# Szyfrowanie częściowo homomorficzne - kryptosystemy PHE

## RSA

Mamy klucz publiczny $(n,e)$ i klucz prywatny $(n,d)$. Szyfrujemy tym samym kluczem publicznym dwie wiadomości $m_1$ i $m_2$. Otrzymane szyfrogramy są postaci:
$$c_1=m_1^e\mod n\qquad c_2=m_2^e\mod n.$$

Jeżeli teraz spróbujemy zdeszyfrować kluczem prywatnym iloczyn otrzymanych szyfrogramów, to otrzymamy
$$(c_1c_2)^d\mod n=c_1^dc_2^d\mod n=(c_1^d\mod n)(c_2^d\mod n)=m_1m_2$$

Otrzymaliśmy zatem, że
$$D\big(E(m_1)E(m_2)\big)=m_1m_2$$
czyli **RSA jest częściowo homomorficzny ze względu na mnożenie**.

## Ćwiczenie 1.

Korzystając z implementacji RSA z ostatnich zajęć sprawdź czy są limity dla liczby homomorficznych operacji mnożenia (tzn. czy od jakiejś liczby operacji na szyfrogramach zaczynamy otrzymywać błędne deszyfrowanie).

In [1]:
import random
from math import gcd

# Simple primality check
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Generate random prime number
def generate_prime(bits=16):
    while True:
        num = random.getrandbits(bits)
        num |= (1 << bits - 1) | 1  # ensure high bit and odd
        if is_prime(num):
            return num

# Modular inverse using Extended Euclidean algorithm
def mod_inverse(a, m):
    def egcd(a, b):
        if a == 0:
            return b, 0, 1
        else:
            g, y, x = egcd(b % a, a)
            return g, x - (b // a) * y, y
    g, x, _ = egcd(a, m)
    if g != 1:
        raise Exception('Inverse mod does not exist')
    else:
        return x % m

In [2]:
def rsa_keygen(bits=32):
    p = generate_prime(bits // 2)
    q = generate_prime(bits // 2)
    n = p * q
    phi = (p - 1)*(q - 1)
    e = 65537
    if gcd(e, phi) != 1:
        e = 3
    d = mod_inverse(e, phi)
    return (n, e), (n, d)

def rsa_encrypt(m, pubkey):
    n, e = pubkey
    return pow(m, e, n)

def rsa_decrypt(c, privkey):
    n, d = privkey
    return pow(c, d, n)

# Check RSA homomorphic multiplication limit
pubkey, privkey = rsa_keygen()
msg = 2
c = rsa_encrypt(msg, pubkey)
ciphertext_product = c

for i in range(1, 100):
    ciphertext_product = (ciphertext_product * c) % pubkey[0]
    decrypted = rsa_decrypt(ciphertext_product, privkey)
    if decrypted != pow(msg, i+1, pubkey[0]):
        print(f"RSA: błędne deszyfrowanie po {i} operacjach mnożenia.")
        break
else:
    print("RSA: 100 operacji mnożenia zakończyło się poprawnie.")

RSA: 100 operacji mnożenia zakończyło się poprawnie.


## Kryptosystem Pailliera

### Generowanie kluczy
- wybieramy losowo liczby pierwsze $p$, $q$ o tej samej długości w zapisie dziesiętnym i obliczamy $n=pq$, $g=n+1$, $\lambda=\phi(n)=(p-1)(q-1)$ oraz  $\mu =\phi (n)^{-1}{\mod {n}}$.
- kluczem publicznym jest $(n,g)$ a kluczem prywatnym $(\lambda,\mu)$


### Szyfrowanie
Szyfrujemy liczbę $0\leq m<n$. Wybieramy losowe $r<n$ względnie pierwsze z $n$ i obliczamy szyfrogram $$c=g^{m}\cdot r^{n}{\mod {n}}^{2}.$$


### Deszyfrowanie
Obliczamy $$m=\mu L(c^{\lambda }{\bmod {n}}^{2}){\bmod {n}},$$ gdzie  $L(x)$ to największa liczba naturalna $\nu$ taka, że $x-1\geq \nu n$.

Pomimo czynnika losowego przy szyfrowaniu, kryptosystem Pailliera ma własność $$D\big(E(m_{1},r_{1})\cdot E(m_{2},r_{2})\big)=m_{1}+m_{2}$$
a zatem jest **częściowo homomorficzny ze względu na dodawanie**.

## Ćwiczenie 2.

Zaimplementuj kryptosystem Pailliera. Sprawdź, czy są limity homomorficznych operacji dodawania.

In [3]:
def L(x, n):
    return (x - 1) // n

def paillier_keygen(bits=32):
    p = generate_prime(bits // 2)
    q = generate_prime(bits // 2)
    n = p * q
    g = n + 1
    lambd = (p - 1)*(q - 1)
    mu = mod_inverse(lambd, n)
    return (n, g), (lambd, mu)

def paillier_encrypt(m, pubkey):
    n, g = pubkey
    r = random.randrange(1, n)
    while gcd(r, n) != 1:
        r = random.randrange(1, n)
    c = (pow(g, m, n*n) * pow(r, n, n*n)) % (n*n)
    return c

def paillier_decrypt(c, privkey, pubkey):
    n, _ = pubkey
    lambd, mu = privkey
    u = pow(c, lambd, n*n)
    return (L(u, n) * mu) % n

# Check Paillier homomorphic addition limit
pubkey_p, privkey_p = paillier_keygen()
c_sum = paillier_encrypt(1, pubkey_p)

for i in range(1, 100):
    c_sum = (c_sum * paillier_encrypt(1, pubkey_p)) % (pubkey_p[0]**2)
    decrypted_sum = paillier_decrypt(c_sum, privkey_p, pubkey_p)
    if decrypted_sum != i + 1:
        print(f"Paillier: błędne deszyfrowanie po {i} operacjach dodawania.")
        break
else:
    print("Paillier: 100 operacji dodawania zakończyło się poprawnie.")

Paillier: 100 operacji dodawania zakończyło się poprawnie.


Kryptosystem nazywamy *w pewnym sensie homomorficznym*, jeżeli operacjami na samych szyfrogramach $E(m_1),...,E(m_k)$ jesteśmy w stanie obliczyć pewne określone kombinacje dodawania i mnożenia oryginalnych wiadomości $m_1,...,m_k$.

## Ćwiczenie 3.

Wykorzystując RSA i Pailliera zaimplementuj kryptosystem, który będzie w stanie obliczyć $(m_1m_2+m_3)\cdot m_4$. Wszystkie działania muszą się odbywać na danych zaszyfrowanych (nie możemy ujawnić chmurze żadnej wiadomości $m_1$, $m_2$, $m_3$, $m_4$). Czy jesteśmy w stanie w ten sposób skonstruować kryptosystem w pewnym sensie homomorficzny?

Tak, jest to kryptosystem częściowo homomorficzny. RSA pozwala wykonywać mnożenie na zaszyfrowanych danych, a Paillier umożliwia dodawanie. Jednak przejście między nimi wymaga częściowego odszyfrowania danych, więc nie jest to pełna homomorficzność – operacje nie są możliwe całkowicie bez odszyfrowywania wyników pośrednich.

In [4]:
def secure_computation(m1, m2, m3, m4):
    # RSA (multiplication)
    rsa_pub, rsa_priv = rsa_keygen()
    c1 = rsa_encrypt(m1, rsa_pub)
    c2 = rsa_encrypt(m2, rsa_pub)
    c_mult = (c1 * c2) % rsa_pub[0]
    decrypted_mult = rsa_decrypt(c_mult, rsa_priv)

    # Paillier (addition)
    paillier_pub, paillier_priv = paillier_keygen()
    c_add_1 = paillier_encrypt(decrypted_mult, paillier_pub)
    c_add_2 = paillier_encrypt(m3, paillier_pub)
    c_added = (c_add_1 * c_add_2) % (paillier_pub[0]**2)
    decrypted_added = paillier_decrypt(c_added, paillier_priv, paillier_pub)

    # Final RSA multiplication
    c_final = rsa_encrypt(decrypted_added, rsa_pub)
    c4 = rsa_encrypt(m4, rsa_pub)
    c_final_mult = (c_final * c4) % rsa_pub[0]
    final_result = rsa_decrypt(c_final_mult, rsa_priv)

    return final_result

# Test the full homomorphic secure operation
m1, m2, m3, m4 = 3, 4, 5, 6  # Example messages
expected_result = (m1*m2 + m3)*m4
computed_result = secure_computation(m1, m2, m3, m4)

print("Oczekiwany wynik:", expected_result)
print("Wynik z homomorficznej operacji:", computed_result)

if computed_result == expected_result:
    print("Operacja homomorficzna zakończyła się sukcesem!")
else:
    print("Błąd w obliczeniach homomorficznych.")

Oczekiwany wynik: 102
Wynik z homomorficznej operacji: 102
Operacja homomorficzna zakończyła się sukcesem!
